In [1]:
import os
import pandas as pd
from pathlib import Path
from dataclasses import dataclass

C:\Users\roshi\AppData\Local\Temp\ipykernel_21784\4249437511.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
os.chdir("../")
%pwd

'c:\\Users\\roshi\\OneDrive\\Desktop\\Git-1\\Text_Classification_ML_NLP'

In [3]:
@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_tfidf: Path
    test_tfidf: Path
    train_y: Path
    test_y: Path
    model_name: str

In [4]:
from src.text_Classification_ML.constants import *
from src.text_Classification_ML.utils.common import *

In [5]:
class ConfigurationManager:
    def __init__(
            self,
            config_file_path = Config_File_Path
            ):
        self.config = read_yaml(config_file_path)

        create_directories([self.config.artifacts_root])

    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir= config.root_dir,
            train_tfidf= config.train_tfidf,
            test_tfidf= config.test_tfidf,
            train_y= config.train_y,
            test_y= config.test_y,
            model_name= config.model_name
        )

        return model_trainer_config

In [6]:
from sklearn.linear_model import LogisticRegression
import scipy.sparse
import joblib

In [7]:
class ModelTrainer:
    def __init__(self,config: ModelTrainerConfig):
        self.config = config


    def train(self):
        # load the matrix
        train_tfidf = scipy.sparse.load_npz(self.config.train_tfidf)
        test_tfidf = scipy.sparse.load_npz(self.config.test_tfidf)
        train = pd.read_csv(self.config.train_y)
        test = pd.read_csv(self.config.test_y)

        # Train a Logistic Regression model
        model = LogisticRegression(max_iter=1000, solver= "liblinear")
       
        model.fit(train_tfidf, train['Rating'])

        joblib.dump(model, os.path.join(self.config.root_dir, self.config.model_name))

In [8]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer = ModelTrainer(config= model_trainer_config)
    model_trainer.train()
except Exception as e:
    raise e